In [13]:
using OhMyJulia
using DataFrames

## Setup

In [65]:
function parse_path(file)
    pathdict = Dict{Pair{String, String}, Vector{Pair{Vector{Pair{String, String}}, Float64}}}()
    
    cp = nothing
    for line in eachline(file)
        m = match(r"([hs]\d+)\s*->\s*([hs]\d+)", line)
        if m != nothing
            pathdict[car(m) => cadr(m)] = []
            cp = pathdict[car(m) => cadr(m)]
            continue
        end
        
        s = split(line, '@')
        if length(s) == 2
            p = parse(car(s))
            p = map(x->Pair(string.(x.args)...), p.args)
            w = parse(cadr(s))
            push!(cp, p => w)
        end
    end
    
    pathdict
end

function parse_topo(file)
    nodes = Set{String}()
    edges = Set{Pair{String, String}}()
    for line in eachline(file)
        m = match(r"([hs]\d+)\s*->\s*([hs]\d+)", line)
        m == nothing && continue
        
        src, dst = car(m), cadr(m)
        push!(nodes, src, dst)
        push!(edges, src => dst)
    end
    collect(nodes), collect(edges)
end

function mkdf(names...)
    DataFrame([[] for i in names], [Symbol(i) for i in names])
end

function cut_path!()
    ref = Dict(k=>length(v) for (k, v) in paths["custom_0"])
    
    for (algo, path) in paths
        for (pair, scheme) in path
            scheme = sort(scheme, rev=true, by=cadr)
            length(scheme) <= ref[pair] && continue
            
            newscheme = scheme[1:ref[pair]]
            factor = sum(cadr.(newscheme))
            path[pair] = [p => w / factor for (p, w) in newscheme]
        end
    end
end

data = "jelly16"
nodes, edges = parse_topo("data/topologies/$data.dot")
core_edges = filter(x->startswith(car(x), "s") && startswith(cadr(x), "s"), edges)
# algos = [basename(x) for x in all_files("data/results/$data/paths") if match(r"_\d+$", x) != nothing]
algos = [basename(x) for x in all_files("data/results/$data/paths") if endswith(x, "_0")]
paths = Dict(algo => parse_path("data/results/$data/paths/$algo") for algo in algos)
cut_path!()

## avg path length

In [70]:
df = mkdf("algorithm", "average length", "max length")
for (algo, path) in paths
    lens = [length(car(p)) for (k, v) in path for p in v]
    push!(df, [algo[1:end-2] mean(lens) maximum(lens)])
end
df

,algorithm,average length,max length
1,custom,5.13472,7
2,semimcfvlb,4.90417,8
3,raeke,4.66528,6
4,semimcfecmp,4.20526,5
5,semimcfraeke,4.66389,6
6,ksp,4.63611,6
7,vlb,4.91528,8
8,semimcfksp,4.65833,6
9,semimcfcustom,5.13472,7
10,semimcfedksp,4.72778,6


## one edge failure

In [67]:
df = mkdf("algorithm", "#path", "affected path", "affected path / #edge", "affected path / #edge / #path", "affected pair", "affected weight", "average affected path", "a1", "a2", "a3", "a4", "a5+")
for (algo, path) in paths
    aweight, apaths = 0, []
    
    for edge in core_edges
        for (pair, scheme) in path
            pair_affected_path = 0
            
            for (p, w) in scheme @when edge in p
                pair_affected_path += 1
                aweight += w
            end
            
            pair_affected_path > 0 && push!(apaths, pair_affected_path)
        end
    end
    
    push!(df, [algo[1:end-2] sum(length∘cadr, path) sum(apaths) sum(apaths)/length(edges) sum(apaths)/length(edges)/sum(length∘cadr, path) length(apaths) aweight mean(apaths) sum(apaths .== 1) sum(apaths .== 2) sum(apaths .== 3) sum(apaths .== 4) sum(apaths .>= 5)])
end
df

,algorithm,#path,affected path,affected path / #edge,affected path / #edge / #path,affected pair,affected weight,average affected path,a1,a2,a3,a4,a5+
1,custom,720,2257,23.5104,0.0326534,2257,752.333,1.0,2257,0,0,0,0
2,semimcfvlb,720,2091,21.7813,0.0302517,1979,540.537,1.05659,1867,112,0,0,0
3,raeke,720,1919,19.9896,0.0277633,1844,552.652,1.04067,1769,75,0,0,0
4,semimcfecmp,380,838,8.72917,0.0229715,796,466.0,1.05276,758,34,4,0,0
5,semimcfraeke,720,1918,19.9792,0.0277488,1831,561.168,1.04752,1744,87,0,0,0
6,ksp,720,1898,19.7708,0.0274595,1712,632.667,1.10864,1530,178,4,0,0
7,vlb,720,2099,21.8646,0.0303675,1970,600.197,1.06548,1842,127,1,0,0
8,semimcfksp,720,1914,19.9375,0.027691,1733,561.463,1.10444,1555,175,3,0,0
9,semimcfcustom,720,2257,23.5104,0.0326534,2257,605.824,1.0,2257,0,0,0,0
10,semimcfedksp,720,1964,20.4583,0.0284144,1964,552.657,1.0,1964,0,0,0,0


## two edges failure

In [68]:
df = mkdf("algorithm", "#path", "affected path", "affected path / #edge", "affected path / #edge / #path", "affected pair", "affected weight", "average affected path", "a1", "a2", "a3", "a4", "a5+")

for (algo, path) in paths
    aweight, apaths = 0, []
    
    for e1 in core_edges, e2 in core_edges @when e1 != e2
        for (pair, scheme) in path
            pair_affected_path = 0
            
            for (p, w) in scheme @when e1 in p || e2 in p
                pair_affected_path += 1
                aweight += w
            end
            
            pair_affected_path > 0 && push!(apaths, pair_affected_path)
        end
    end
    
    push!(df, [algo[1:end-2] sum(length∘cadr, path) sum(apaths) sum(apaths)/length(edges) sum(apaths)/length(edges)/sum(length∘cadr, path) length(apaths) aweight mean(apaths) sum(apaths .== 1) sum(apaths .== 2) sum(apaths .== 3) sum(apaths .== 4) sum(apaths .>= 5)])
end
df

,algorithm,#path,affected path,affected path / #edge,affected path / #edge / #path,affected pair,affected weight,average affected path,a1,a2,a3,a4,a5+
1,custom,720,278764,2903.79,4.03304,264996,92921.2,1.05196,251228,13768,0,0,0
2,semimcfvlb,720,258716,2694.96,3.743,234756,67242.5,1.10206,211248,23056,452,0,0
3,raeke,720,238152,2480.75,3.44549,219702,68744.7,1.08398,201570,17814,318,0,0
4,semimcfecmp,380,104388,1087.38,2.86151,96922,58164.0,1.07703,90154,6070,698,0,0
5,semimcfraeke,720,238032,2479.5,3.44375,218256,69790.1,1.09061,198856,19024,376,0,0
6,ksp,720,235632,2454.5,3.40903,205010,78544.0,1.14937,175674,28050,1286,0,0
7,vlb,720,259686,2705.06,3.75703,233762,74472.7,1.1109,208520,24560,682,0,0
8,semimcfksp,720,237558,2474.56,3.43689,207370,69802.2,1.14558,178338,27876,1156,0,0
9,semimcfcustom,720,278764,2903.79,4.03304,264996,75172.2,1.05196,251228,13768,0,0,0
10,semimcfedksp,720,243552,2537.0,3.52361,233044,68755.5,1.04509,222536,10508,0,0,0


## 3 edges failure

In [69]:
df = mkdf("algorithm", "#path", "affected path", "affected path / #edge", "affected path / #edge / #path", "affected pair", "affected weight", "average affected path", "a1", "a2", "a3", "a4", "a5+")

for (algo, path) in paths
    aweight, apaths = 0, []
    
    for e1 in core_edges, e2 in core_edges, e3 in core_edges @when e1 != e2 != e3
        for (pair, scheme) in path
            pair_affected_path = 0
            
            for (p, w) in scheme @when e1 in p || e2 in p || e3 in p
                pair_affected_path += 1
                aweight += w
            end
            
            pair_affected_path > 0 && push!(apaths, pair_affected_path)
        end
    end
    
    push!(df, [algo[1:end-2] sum(length∘cadr, path) sum(apaths) sum(apaths)/length(edges) sum(apaths)/length(edges)/sum(length∘cadr, path) length(apaths) aweight mean(apaths) sum(apaths .== 1) sum(apaths .== 2) sum(apaths .== 3) sum(apaths .== 4) sum(apaths .>= 5)])
end
df

LoadError: [91mInterruptException:[39m

## one node failure

In [20]:
df = mkdf("algorithm", "affected path", "affected pair", "affected weight")
for (algo, path) in paths
    apath, apair, aweight = 0, 0, 0
    
    for node in nodes @when startswith(node, "s")
        for (pair, scheme) in path
            pair_affected = false
            
            for (p, w) in scheme @when any(x->node in x, p)
                pair_affected = true
                apath += 1
                aweight += w
            end
            
            apair += pair_affected
        end
    end
    
    push!(df, [algo[1:end-2] apath apair aweight])
end
df

,algorithm,affected path,affected pair,affected weight
1,custom,2977,2001,992.332
2,semimcfvlb,2811,1739,780.537
3,raeke,2639,1604,792.652
4,semimcfecmp,1218,896,706.0
5,semimcfraeke,2638,1591,801.168
6,ksp,2618,1467,872.666
7,vlb,2819,1728,840.197
8,semimcfksp,2618,1467,783.613
9,semimcfcustom,2977,2001,845.824
10,semimcfedksp,2684,1722,792.657


## throuput

In [143]:
host_list = readlines("data/hosts/$data.hosts")
demand_list = map(readlines("data/demands/$data.txt")) do line
    line = map(parse, split(line, ' '))
    reshape(line, length(host_list), length(host_list))'
end

f(x) = x < 0.333 ? x :
       x < 0.667 ? 0.333 + 3(x-0.333) :
       x < 0.9   ? 1.333 + 10(x-0.667) :
       x < 1.0   ? 3.666 + 70(x-0.9) :
       x < 1.1   ? 10.666 + 500(x-1.0) :
       60.666 + 5000(x-1.1)

df = mkdf("algorithm", "mean(l/c)", "sum(f(l/c))", "sum((l/c)^2)")
for (algo, path) in paths
    total_flow = Dict(edge => 0. for edge in edges if startswith(car(edge), "s") && startswith(cadr(edge), "s"))
    
    for demands in demand_list[1:1]
        for (pair, scheme) in path
            demand = demands[parse(Int, cdr(car(pair))), parse(Int, cdr(cadr(pair)))]
            
            for (p, w) in scheme, edge in p @when startswith(car(edge), "s") && startswith(cadr(edge), "s")
                total_flow[edge] += w * demand
            end
        end
    end
                
    tf = collect(values(total_flow)) ./ 2^30
    
    push!(df, [algo[1:end-2] mean(tf) sum(f, tf) sum(tf .^ 2)])
end
df

,algorithm,mean(l/c),sum(f(l/c)),sum((l/c)^2)
1,custom,0.276464,10.6718,3.06417
2,semimcfvlb,0.245018,8.71602,2.35285
3,raeke,0.234088,8.35385,2.2771
4,semimcfecmp,0.194276,6.29304,1.65583
5,semimcfraeke,0.260101,9.30472,2.54834
6,ksp,0.255224,9.82557,2.79309
7,vlb,0.223425,8.18249,2.25152
8,semimcfksp,0.275418,9.69963,2.70834
9,semimcfcustom,0.227599,7.8902,2.0846
10,semimcfedksp,0.227599,7.8949,2.08631


In [12]:
function gen_gravity(data, α=2^26, n=4)
    hosts = readlines("data/hosts/$data.hosts")
    weigths = map(x->rand(), hosts)
    open("data/demands/$data.gravity", "w") do fout
        for l in 1:n
            list = [ α * weigths[i] * weigths[j]
                     for i in 1:length(hosts)
                     for j in 1:length(hosts) ]
            println(fout, join(list, ' '))
        end
    end
end
gen_gravity("Esnet")